## **Challenge 33: Implement Diffie-Hellman**

In [1]:
import secrets
# 
import src.dh as dh
import src.sha256 as sha256
import src.convert as convert

In [2]:
prime = 37
generator = 5
print(f"Diffie-Hellman with parameters: \n- Prime: {prime}\n- Generator: {generator}\n")

# Generate alice and bob's private and public keys
private_key_a, public_key_a = dh.gen_keypair(prime, generator)
private_key_b, public_key_b = dh.gen_keypair(prime, generator)
print(f"Alice's Private Key: {private_key_a}, Public Key: {public_key_a}")
print(f"Bob's Private Key: {private_key_b}, Public Key: {public_key_b}\n")

# Generate alice and bob's shared secrets (should be the same)
secret_a = dh.shared_secret(prime, public_key_a, private_key_b)
secret_b = dh.shared_secret(prime, public_key_b, private_key_a)
print(f"Alice's Secret: {secret_a}")
print(f"Bob's Secret: {secret_b}")

Diffie-Hellman with parameters: 
- Prime: 37
- Generator: 5

Alice's Private Key: 19, Public Key: 32
Bob's Private Key: 21, Public Key: 23

Alice's Secret: 14
Bob's Secret: 14


In [15]:
prime = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
generator = 2
print(f"Diffie-Hellman with parameters: \n- Prime: {prime}\n- Generator: {generator}\n")

# Generate alice and bob's private and public keys
private_key_a, public_key_a = dh.gen_keypair(prime, generator)
private_key_b, public_key_b = dh.gen_keypair(prime, generator)
print(f"Alice's Private Key: {private_key_a}, Public Key: {public_key_a}")
print(f"Bob's Private Key: {private_key_b}, Public Key: {public_key_b}\n")

# Generate alice and bob's shared secrets (should be the same)
secret_a = dh.shared_secret(prime, public_key_a, private_key_b)
secret_b = dh.shared_secret(prime, public_key_b, private_key_a)
print(f"Alice's Secret: {secret_a}")
print(f"Bob's Secret: {secret_b}")

Diffie-Hellman with parameters: 
- Prime: 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
- Generator: 2

Alice's Private Key: 1937151472924568843374232080577698349737276895241149116787603800922163864562677661087161056336095398531545458603403416758327728705779341551954265239944779055834408162899806034480472584876008249657592062613147858482143771945540355191373306939609610603921670975657352243770217942843628640669195971853509251388571272990289382804785019276592764985829659276857530834871860271507865760364969425662785020285617091300179005771358766142222071633225854173133940952276

In [16]:
# Generate a 128 bit key from the hash of the shared secret
key = sha256.digest(convert.bigint2bytes(secret_a))[:16]
print(f"\n128 bit key: {key} ({len(key)} bytes)")


128 bit key: b'\xe6\x90\x96\xaf\xed@\xdf\x17\xf4\xb8#\xa3\x9d\xe56\xa5' (16 bytes)


## **Challenge 34: Implement a MITM key-fixing attack on Diffie-Hellman with parameter injection**